In [ ]:
import os, sys
import comtypes.client

In [ ]:
# coords = np.array([]);
# elems = np.array([]);
# apoyos = np.array([]);
# loads = 

# Coneccion: 1 para sap2000 y 2 para etabs
connect_to = 2;
# True: para anjuntar a etabs o sap200 en ejecucion.
rutaEspecifica = False

In [ ]:
smodel, ret = None, None
conn = {};
if connect_to: # SAP2000
    conn = {
        'app_ruta': "C:\\Program Files\\Computers and Structures\\SAP2000 23\\SAP2000.exe",
        'app_adjunto': "CSI.SAP2000.API.SapObject",
        'app_helper': 'SAP2000v1.Helper',
        'app_helper_query': comtypes.gen.SAP2000v1.cHelper,
        }
elif (connect_to==2): # ETABS
    conn = {
        'app_ruta': "C:\\Program Files\\Computers and Structures\\ETABS 21\\ETABS.exe",
        'app_adjunto': "CSI.ETABS.API.ETABSObject",
        'app_helper': "ETABSv1.Helper",
        'app_helper_query': comtypes.gen.ETABSv1.cHelper,
        }
else: 
    print("No tenemos soporte ...")
    sys.exit()

try:
    connect_to_app = comtypes.client.GetActiveObject(conn['app_adjunto'])
    print("Coneccion exitosa!.\nadjuntando a una instancia existente.")
except (OSError, comtypes.COMError):
    print("No se encontró ninguna instancia en ejecución del programa(Etabs).")

    print("Tratando de Ejecutar etabs!.")
    #create API helper object
    helper = comtypes.client.CreateObject(conn['app_helper'])
    helper = helper.QueryInterface(conn['app_helper_query'])
    # helper = comtypes.client.CreateObject(conn['app_helper']).QueryInterface(conn['app_helper_query'])
    if rutaEspecifica: # para versiones anteriores a la ultima version instalada
        ProgramPath = conn['app_ruta'];
        try:
            connect_to_app = helper.CreateObject(ProgramPath)
            print("Coneccion establecida!.\nConexion Manual")
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            connect_to_app = helper.CreateObjectProgID(conn['app_adjunto']) 
            print("Coneccion establecida!.")
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)
    print("Ejecutando etabs!.  ultima version instalada o existente")
    #start ETABS application | ejecutar la Aplicacion ETABS.
    connect_to_app.ApplicationStart()

smodel = connect_to_app.SapModel;

## Creacion de nuevo modelo

In [ ]:
# ------------------- creacion de nuevo modelo ------------------
smodel.SetModelIsLocked(False)

kN_m_C=6
# string Name
# eMatType MatType
# int Color = -1
# string Notes = ""
# string GUID = ""
# smodel.SetPresentUnits(kN_m_C)
smodel.InitializeNewModel(kN_m_C)

ret = smodel.File.NewBlank()
ret

In [ ]:
frameName = ""
point1 = ""
point2 = ""
# ret = ""
[FrameName1, ret] = smodel.FrameObj.AddByCoord(0, 0, 0, 0, 0, 3, frameName);# True,"Defualt","","Global"
[FrameName1, ret] = smodel.FrameObj.AddByCoord(6, 0, 0, 6, 0, 3, frameName);
[FrameName1, ret] = smodel.FrameObj.AddByCoord(0, 0, 3, 6, 0, 3, frameName);

restr = [True, True, True, True, True, True];
smodel.Pointobj.SetRestraint(point1, restr);
smodel.Pointobj.SetRestraint(point2, restr);

lpName = "Test";
# 1 = Force per unit length
# 2 = Moment per unit length
# smodel.FrameObj.SetLoadDistributed(FrameName, "DEAD", 1, 10, 0.0, 1.0, 15.0, 5.0);
smodel.FrameObj.SetLoadDistributed(frameName, lpName, 1, 10, 0.0, 1.0, 15.0, 5.0);

smodel.View.RefreshView(0, False);

# Save model
#full path to the model, set it to the desired path of your model
APIPath = 'C:\\CSi_API_Example'
if not os.path.exists(APIPath):
    try: os.makedirs(APIPath)
    except OSError: pass
ModelPath = APIPath + os.sep + 'Frame3D.edb'
smodel.File.Save(ModelPath);

In [ ]:
# ------------------ Analisis y ejecucion del modelo -----------------
# # una ves ejecutado una primera vez ya se peude 
# # comentar de la linia 26 a 56 y descomentar 60 a 63
# smodel.SetModelIsLocked(false);
# for i in range(3):
#     smodel.FrameObj.SetEndLengthOffset(str(i), flase, 0.0, 0.0, 0.0);

# 'run analysis
smodel.Analyze.RunAnalysis();

In [ ]:
# 'deselect all cases and combos
ret = smodel.Results.Setup.DeselectAllCasesAndCombosForOutput();
# 'check if case is selected
ret = smodel.Results.Setup.SetCaseSelectedForOutput(lpName);

# 'get frame forces
name=""
eItemElm = 0
numberResults = 0;
obj = [];
objSta = [];
elm = [];
elmSta = [];
loadCase = [];
stepType = [];
stepNum = [];
p = [];
v2 = [];
v3 = [];
t = [];
m2 = [];
m3 = [];
[name, eItemElm, numberResults, 
    obj, objSta, elm, elmSta, loadCase, stepType, stepNum,
    p, v2, v3, t, m2] = smodel.Results.FrameForce("All", eItemElm, numberResults,
    obj, objSta, elm, elmSta, loadCase, stepType, stepNum,
    p, v2, v3, t, m2, m3);

# 
print("Mesage de salida\nmesnage1\tmensage2\tmensage3");
for i in range(ret[2]):
    print(f"{obj[i]}\t{[i]}\t{ret[15][i]}");


In [ ]:
# Close the program | Cerrar la aplicacio
if(connect_to_app.ApplicationExit(False)): print("No se pudo cerrar la coneccion!")
else: print("Se cerro con exito.")
    
onnect_to_app, smodel = None, None
del (connect_to_app, smodel)